# Forest Fires in Brazil Analysis

Recently a lot has been said about the fires that have outbreaked in Brazil, more specifically in the Amazon Forest Area. This Analysis has the ojective to explore the dataset and see what conclusions can be obtained and it also has the objective to serve as a practice exercise to data analysis.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/forest-fires-in-brazil/amazon.csv', encoding='Latin-1')

In [ ]:
df.head(10)

The data is organized in the following way: year represent the year when fires happened and month represent in which month they occurred ant number how many occurred in that month.

In [ ]:
df['year'].describe()

Year is a float type this is unecessary, so it will be converted to int

In [ ]:
df['year'] = df['year'].astype(int)

In [ ]:
months = {'Janeiro':'January', 'Fevereiro':'February', 'Março':'March', 'Abril':'April', 'Maio':'May', 
          'Junho':'June', 'Julho':'July', 'Agosto':'August', 'Setembro':'September', 'Outubro':'October',
         'Novembro':'November', 'Dezembro':'December'}
df['month'] = df['month'].map(months)
print(df['month'].unique())

In [ ]:
df['state'].unique()

In [ ]:
df['state'] = df['state'].str.replace('á', 'a')
df['state'] = df['state'].str.replace('Piau', 'Piaui')

Removed accent marks and corrected the name for Piaui.

In [ ]:
df['date'].value_counts()

The dates are wrong, they are all first of january for that year, so they wont be used.

# Yearly Analysis

In the next part the fires will be mostly analyzed by year.

In [ ]:
df_year = df[['year', 'number']]

In [ ]:
df_year = df_year.groupby('year').sum()

In [ ]:
df_year.head()

df_year contains the sum of all fires by year

In [ ]:
import matplotlib.ticker as ticker
sns.set(rc = {'axes.facecolor':'#bbbbbb'})
plt.figure(figsize=(15,4))
sns.set_context(rc = {'lines.linewidth':3})
g = sns.lineplot(x=df_year.index, y='number', data=df_year)
g.xaxis.set_major_locator(ticker.MultipleLocator(1))

Recently the media has given a lot of atention to the Amazon Forest fires, in order to see if the number of fires has risen let's plot the fires per year in the state of Amazonas.

In [ ]:
df_amazonas = df.loc[df['state']=='Amazonas']

In [ ]:
df_amazonas = df_amazonas[['year', 'number']].groupby('year', as_index=False).sum()

In [ ]:
sns.set(rc = {'axes.facecolor':'#bbbbbb'})
plt.figure(figsize=(15,4))
sns.set_context(rc = {'lines.linewidth':3})
g = sns.lineplot(x='year', y='number', data=df_amazonas)
g.xaxis.set_major_locator(ticker.MultipleLocator(1))

Although the data for 2018 and 2019 is unavailable it's safe to say that in the pas few years the fires in Amazonas are't rising.

One could say that the Amazon Forest is bigger than the state of Amazonas and therefore there could be a trend. Even though that would be hard considering that Amazonas should represent the Amazon pretty well, the data for the states that are part of the Amazon Forest will be summed an ploted.

In [ ]:
amazon_states = ['Acre','Amapa', 'Amazonas', 'Rondonia', 'Roraiama', 'Para', 'Tocantins']

The states in the list will be taken into account, there are Amazon areas in Mato Grosso and Maranhão but it isn't the most present biom.
source: https://pt.wikipedia.org/wiki/Amaz%C3%B4nia_Legal (it's in portuguese, here in the division section the Amazon Forest area is well defined)

In [ ]:
filter_am = df['state']=='Acre'
for i in range(1, 7):
    filter_am = filter_am|(df['state']==amazon_states[i])

In [ ]:
df_amstates = df.loc[filter_am, ['year', 'number','state']]

The selection is working.

In [ ]:
df_amstates = df_amstates.groupby('year', as_index=False).sum()

In [ ]:
sns.set(rc = {'axes.facecolor':'#bbbbbb'})
plt.figure(figsize=(15,4))
sns.set_context(rc = {'lines.linewidth':3})
g = sns.lineplot(x='year', y='number', data=df_amstates)
g.xaxis.set_major_locator(ticker.MultipleLocator(1))

As with the Amazonas state there isn's a clear rise in the number of fires in the selected states in recent, even though there was an increase from 2013 to 2016 in 2017 fires decreased. The numbers seem to be quite stable 2002 onwars there were decreases and increases.

# State Analysis

The next part will focus on states.

In [ ]:
df_st = df[['state', 'number']]
df_st = df_st.groupby('state', as_index=False).sum()
df_st_sorted = pd.DataFrame()
df_st_sorted['number'] = df_st['number'].sort_values(ascending=False)
df_st_sorted['state'] = df_st['state']
df_st_sorted = df_st_sorted.dropna()

df_st_sorted contains the sum of fires per state in descending order

In [ ]:
sns.set(rc = {'axes.facecolor':'#bbbbbb'})
g = sns.catplot(x='state', y='number', data=df_st_sorted, kind='bar', ci=None, palette='Oranges_r')
g.fig.set_figwidth(18)
g.fig.set_figheight(8)
g.set_xticklabels(rotation=45)
plt.title('Total Number of Fires per State(1998-2017)', size=16)
plt.xlabel('States')
plt.ylabel('Number of Fires')

This graph shows the states with the most fires, but it for a better comprehension it should be taken into account the area of the state

In [ ]:
state_area = pd.Series(index = ['Mato Grosso', 'Paraiba', 'Sao Paulo', 'Rio', 'Bahia', 'Piaui', 'Goias', 
                                'Minas Gerais', 'Tocantins', 'Amazonas', 'Ceara', 'Maranhao', 'Para',
                               'Pernambuco', 'Roraima', 'Santa Catarina', 'Amapa', 'Rondonia', 'Acre', 
                                'Espirito Santo', 'Alagoas', 'Distrito Federal', 'Sergipe'],
                      data = [903366.192, 56585, 248222.362, 43780.172, 564733.177, 251577.738, 340111.783,
                            586522.122, 277720.520, 1559159.148, 148920.472, 331937.450, 1247954.666,
                            98311.616, 224300.506, 95736.165, 142828.521, 237590.547, 164123.040,
                            46095.583, 27778.506, 5779.999, 21915.116])
                

This is a series containing the area for each state
source:https://pt.wikipedia.org/wiki/Lista_de_unidades_federativas_do_Brasil_por_%C3%A1rea

In [ ]:
for state in df_st_sorted['state']:
    df_st_sorted.loc[df_st_sorted['state']==state, 'state_area'] = state_area[state]
df_st_sorted.head()

In [ ]:
df_st_sorted['fires_per_km2'] = df_st_sorted['number']/df_st_sorted['state_area']

Creating a fires per km squared column.

In [ ]:
df_km_sorted = pd.DataFrame()
df_km_sorted['fires_per_km2'] = df_st_sorted['fires_per_km2'].sort_values(ascending=False)
df_km_sorted['state'] = df_st['state']

df_km_sorted contains the fires per km squared by state in descending order.

In [ ]:
sns.set(rc = {'axes.facecolor':'#bbbbbb'})
g = sns.catplot(x='state', y='fires_per_km2', data=df_km_sorted, kind='bar', ci=None, palette='Oranges_r')
g.fig.set_figwidth(18)
g.fig.set_figheight(8)
g.set_xticklabels(rotation=45)
plt.title('Fires per km2 by State', size=16)
plt.xlabel('States', size=12)
plt.ylabel('Fires per km2 (1998-2017)', size=12)

It is possible to see that proportionally by area, Rio is the state with more fires, and Amazonas and Pará the states with the least fires. Naturally that doesn't take into account the size of the fire, the best way to analyze the fires would be to have the data about how much florest was destroyed by the fires.

# Monthly Analysis

The next part will focus on months.

In [ ]:
df_month = df[['month', 'number']].groupby('month', as_index=False).sum()

In [ ]:
months = pd.Series(df['month'].unique())
df_month_org = pd.DataFrame()
df_month_org['month'] = months
df_month_org['number'] = df_month['number']

In [ ]:
sns.set(rc = {'axes.facecolor':'#bbbbbb'})
g = sns.catplot(x = 'month', y = 'number', data = df_month_org, kind = 'bar', ci = None, palette='hls')
plt.title('Fires per Month', size=16)
plt.ylabel('Number of Fires', size=12)
plt.xlabel('Months', size=12)
g.fig.set_figheight(10)
g.fig.set_figwidth(20)

As it is seen the months of February, June, October and November are the ones with the most fires, in a country the size of Brazil it is necessary to analize each state or to group them by climate in order to see the affect of the months, for example in a certain region the dryest month can be one and in other region the same month may not be particularly dry.

In order to explore the differences between states, the graphs of number of fires per month will be ploted for every brazillian state. That will allow the monthly differences to be spotted easily.

In [ ]:
sns.set(rc = {'axes.facecolor':'#bbbbbb'})
fig = plt.figure(figsize=(16, 80))
states = df['state'].unique()
k = 0
for state in states:
    ax = fig.add_subplot(12, 2, k+1)
    subset = df[(df['state']==state)].groupby('month', as_index = False).sum()
    subset_sorted = pd.DataFrame()
    subset_sorted['month'] = months
    subset_sorted['number'] = subset['number']
    ax.plot(subset_sorted['month'], subset_sorted['number'], c='red')
    plt.title('Number of Fires per Month in {} (1998-2017)'.format(str(state)), size =13, fontweight='demi')
    plt.xticks(rotation=45, size=11, y=0.02)
    plt.ylabel('Number of Fires')
    plt.tick_params(direction="in")
    k+=1
plt.show()

As predicted the months with the most fires aren't the same for every state. This can be correlated to the climate in the state as things like temperature and humidity probably affect the number of fires.

To see if the quantity of fires per month is a general trend the fires per month per year for a determained state in the same grid will be ploted, that way the comparison will be much easier. The state of Mato Grosso was chosen to be the one ploted because it has the most fires.

In [ ]:
df_mt = df.loc[df['state'] == 'Mato Grosso']

In [ ]:
sns.set(rc = {'axes.facecolor':'#bbbbbb'})
fig = plt.figure(figsize=(18,12))
years = [2012, 2013, 2014, 2015, 2016, 2017]
colors = ['red', 'blue', 'green', 'yellow', 'black', 'purple']
#only a few years were chosen in oreder to make the graph with a cleaner look
#color are the color that will be used
k = 0
for year in years:
    subset = df_mt.loc[df_mt['year']==year]
    subset = subset.groupby('month', as_index = False).sum()
    #subset is the dataframe of the year that will be used grouped by month
    subset_sorted = pd.DataFrame()
    subset_sorted['month'] = months
    subset_sorted['number'] = subset['number']
    #subset_sorted puts the months in the correct order
    plt.plot(subset_sorted['month'], subset_sorted['number'], label=year, c=colors[k], linewidth=3)
    k += 1
plt.legend(prop={'size':'17'})
plt.xticks(size=13)
plt.title('Fires per Month in Mato Gross (2012-2017)', size=16)
plt.show()

 By looking at graph for Mato Grosso, it is possible to see that there is a general trend, that is the there is a cycle, even though it's not very precise.

# Conclusion 

 Contrary to popular belief the number of fires in the state of Amazonas did not see a increase in it's fires, even in the Amazon Region it's not clear that the number of fires is rising, even though in 2018 and 2019 the number may have risen. 
 
 By looking at the fires per month and more deeply into the state of Mato Grosso it seems that the fires follow a yearly pattern, i.e. there are months where fires are usually more common than other, which was expected. This pattern seems to vary from state to state, knowledge from the state's respective climate may help in determining why.
 
 Not only that, the distribution of fires is very unequal and Mato Grosso is by a large margin the state with more fires, but by taken into account the area of the state the situation is very different and Rio de Janeiro is the state with the most fires per km squared.